## 1. Import data

In [40]:
! python c:/Users/rahmim00/Documents/Notebooks/Clustering/pc_clustering/src/data/import_data.py

2018-07-17 15:10:22,738 - __main__ - INFO - Load raw data file (Huge file, please be patient)...
2018-07-17 15:10:41,103 - __main__ - INFO - Droping uneccessary columns...
2018-07-17 15:10:41,911 - __main__ - INFO - Setting headers info...
2018-07-17 15:10:44,199 - __main__ - INFO - Loading products descriptions...
2018-07-17 15:10:59,242 - __main__ - INFO - Loading clients descriptions...
2018-07-17 15:10:59,571 - __main__ - INFO - Saving store counts file...
2018-07-17 15:11:01,307 - __main__ - INFO - Saving clients count by product...


## 2. Make dataset

In [1]:
! python c:/Users/rahmim00/Documents/Notebooks/Clustering/pc_clustering/src/data/make_dataset.py 1

2018-08-23 16:48:58,101 - __main__ - INFO - *** Making the final dataset from raw data ***
2018-08-23 16:48:58,102 - __main__ - INFO - loading raw data sales file...
2018-08-23 16:48:58,350 - __main__ - INFO - remove null sales...
2018-08-23 16:48:58,399 - __main__ - INFO - trim empty values...
2018-08-23 16:48:59,002 - __main__ - INFO - shit series to origin with 1 offset and range of 16 ...
2018-08-23 16:48:59,870 - __main__ - INFO - ==> Saving raw state data to p2_raw
2018-08-23 16:48:59,954 - __main__ - INFO - remove rare sales...
2018-08-23 16:48:59,962 - __main__ - INFO - Smoothing the series window = 2
2018-08-23 16:49:00,027 - __main__ - INFO - ==> Saving processed data to p2_clean
2018-08-23 16:49:00,084 - __main__ - INFO - ==> Saving raw values data to p2_series


## 3. Train clustering model

In [29]:
! python c:/Users/rahmim00/Documents/Notebooks/Clustering/pc_clustering/src/models/train_model_clustering.py Autumn --k=12 --version=99

2018-08-29 10:45:01,506 - __main__ - INFO - *** Train the clustering model ***
2018-08-29 10:45:01,623 - __main__ - INFO - Init clustering model
2018-08-29 10:45:01,623 - __main__ - INFO - Training clustering model with 12 clusters
2018-08-29 10:45:02,447 - __main__ - INFO - Saving model to << p2_clusters_Autumn >> 


## 4. Build features

In [38]:
! python c:/Users/rahmim00/Documents/Notebooks/Clustering/pc_clustering/src/features/build_features.py

2018-09-03 14:58:57,554 - __main__ - INFO - *** Build features for classification and prediction models ***
2018-09-03 14:58:57,554 - __main__ - INFO - Load raw sales file...
2018-09-03 14:58:57,572 - __main__ - INFO - Load product description file...
2018-09-03 14:58:57,993 - __main__ - INFO - Load clients count  file...
2018-09-03 14:58:58,000 - __main__ - INFO - Load number of clients by week...
2018-09-03 14:58:58,031 - __main__ - INFO - Feature engineering...
2018-09-03 14:58:58,275 - __main__ - INFO - ==> Saving features file to <<clf_features>> ...
2018-09-03 14:58:58,326 - __main__ - INFO - Dataset  6815  succefully made !
2018-09-03 14:58:58,326 - __main__ - INFO - Creating encoders...


## 5. Train classifier

In [39]:
! python c:/Users/rahmim00/Documents/Notebooks/Clustering/pc_clustering/src/models/train_model_classifier.py Autumn --version=1

2018-09-03 14:59:02,992 - __main__ - INFO - *** Train the classifier to predict clusters ***
2018-09-03 14:59:02,992 - __main__ - INFO - Loading data file...
2018-09-03 14:59:03,020 - __main__ - INFO - Preparing data...
2018-09-03 14:59:03,022 - __main__ - INFO - Encoding data...
2018-09-03 14:59:03,042 - __main__ - INFO - Training classifier...
2018-09-03 14:59:03,219 - __main__ - INFO - Saving classifier model to << classifier_Autumn_v1.pkl >>...


## 6. Train Predictor

In [34]:
! python c:/Users/rahmim00/Documents/Notebooks/Clustering/pc_clustering/src/models/train_model_estimator.py --version=99

'DataFrame' object has no attribute 'Nstore'


2018-08-29 10:47:29,821 - __main__ - INFO - *** Train the estimator model ***
2018-08-29 10:47:29,821 - __main__ - INFO - Loading data file...


## 7. New Product classification

In [8]:
! python c:/Users/rahmim00/Documents/Notebooks/Clustering/pc_clustering/src/features/get_features.py

2018-07-19 10:57:29,601 - __main__ - INFO - *** Build features for New Product Intorduction ***
2018-07-19 10:57:29,602 - __main__ - INFO - Load new products file...
2018-07-19 10:57:29,616 - __main__ - INFO - Load clients count  file...
2018-07-19 10:57:29,620 - __main__ - INFO - Feature engineering...
2018-07-19 10:57:29,640 - __main__ - INFO - ==> Saving features file to new_clf_features ...
2018-07-19 10:57:29,640 - __main__ - INFO - Dataset 7 succefully made !


In [25]:
! python c:/Users/rahmim00/Documents/Notebooks/Clustering/pc_clustering/src/models/predict_model_classifier.py